In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
import matplotlib.pyplot as plt

In [28]:
unames=['user_id','gender','age','occupation','zip']
rnames=['user_id','movie_id','ratings','timestamp']
mnames=['movie_id','title','genres']

upath = r"C:\Users\Administrator\movie_data\movielens\users.dat"
rpath = r"C:\Users\Administrator\movie_data\movielens\ratings.dat"
mpath = r"C:\Users\Administrator\movie_data\movielens\movies.dat"


users = pd.read_csv(upath,sep="::", header=None, names = unames)
ratings = pd.read_csv(rpath,sep="::", header=None, names = rnames)
movies = pd.read_csv(mpath,sep="::", header=None, names = mnames)


In [30]:
# 전체 3개를 모두 하나로 묶기 위해서는
# 공통된 부분이 만들어 질 수 있게
# unames와 rname 를 user_id 를 기준으로 먼저 묶고 나서
# movie_id를 기준으로 mnames와 함께 묶어준 것입니다.
data = pd.merge(pd.merge(ratings,users),movies)
data

,user_id,movie_id,ratings,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...,...,...,...,...
1000204,5949,2198,5,958846401,M,18,17,47901,Modulations (1998),Documentary
1000205,5675,2703,3,976029116,M,35,14,30030,Broken Vessels (1998),Drama
1000206,5780,2845,1,958153068,M,18,17,92886,White Boys (1999),Drama
1000207,5851,3607,5,957756608,F,18,20,55410,One Little Indian (1973),Comedy|Drama|Western


In [102]:
"""
행 인덱스로 title이 오고
열 인덱스는 성별에 따라 나뉘고 
평점에 대한 평균이 나올 수 있도록
"""

mean_ratings = data.pivot_table(values='ratings', index='title',
                             columns='gender', aggfunc=np.mean)
mean_ratings

gender,F,M
title,,
"$1,000,000 Duck (1971)",3.375000,2.761905
'Night Mother (1986),3.388889,3.352941
'Til There Was You (1997),2.675676,2.733333
"'burbs, The (1989)",2.793478,2.962085
...And Justice for All (1979),3.828571,3.689024
...,...,...
"Zed & Two Noughts, A (1985)",3.500000,3.380952
Zero Effect (1998),3.864407,3.723140
Zero Kelvin (Kj�rlighetens kj�tere) (1995),NaN,3.500000


In [103]:
"""
영화 평점에 부여한 건수가 많아야 의미가 
있는 평균 점수가 만들어질 것이다.
평점을 부여한 사람이 1명 혹은 2명이라면 
그 평점이 적합하다고 보기 어려울 것이다.
=>>
영화 별 평점을 매긴 사람들의 수를 구분해 봐야할 것이다.
"""

ratings_by_title = data.groupby('title').size()
active_titles = ratings_by_title[ratings_by_title >= 250]
active_titles

title
'burbs, The (1989)                   303
10 Things I Hate About You (1999)    700
101 Dalmatians (1961)                565
101 Dalmatians (1996)                364
12 Angry Men (1957)                  616
                                    ... 
Young Guns (1988)                    562
Young Guns II (1990)                 369
Young Sherlock Holmes (1985)         379
Zero Effect (1998)                   301
eXistenZ (1999)                      410
Length: 1216, dtype: int64

In [104]:
# 평점을 기준 이상 받은 애들의 평균 평점
act_mean_ratings = mean_ratings.loc[active_titles.index]
act_mean_ratings.sort_values(by='F', ascending=False)

gender,F,M
title,,
"Close Shave, A (1995)",4.644444,4.473795
"Wrong Trousers, The (1993)",4.588235,4.478261
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650,4.464589
Wallace & Gromit: The Best of Aardman Animation (1996),4.563107,4.385075
Schindler's List (1993),4.562602,4.491415
...,...,...
"Avengers, The (1998)",1.915254,2.017467
Speed 2: Cruise Control (1997),1.906667,1.863014
Rocky V (1990),1.878788,2.132780


In [107]:
"""
남녀간 평점차를 구했을 때 
가장 평점차가 큰 영화 제목이 
무엇인지 확인하기
"""
act_mean_ratings['diff'] = act_mean_ratings.F - act_mean_ratings.M
sorted_by_diff = act_mean_ratings.sort_values(by='diff')
sorted_by_diff[::1]

gender,F,M,diff
title,,,
"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,-0.726351
"Kentucky Fried Movie, The (1977)",2.878788,3.555147,-0.676359
Dumb & Dumber (1994),2.697987,3.336595,-0.638608
"Longest Day, The (1962)",3.411765,4.031447,-0.619682
"Cable Guy, The (1996)",2.250000,2.863787,-0.613787
...,...,...,...
Steel Magnolias (1989),3.901734,3.365957,0.535777
Little Women (1994),3.870588,3.321739,0.548849
Grease (1978),3.975265,3.367041,0.608224


In [122]:
rs_by_title = data.groupby('title')['ratings'].std()
rs_by_title = rs_by_title.loc[active_titles.index]
rs_by_title.sort_values(ascending=False)

title
Dumb & Dumber (1994)                     1.321333
Blair Witch Project, The (1999)          1.316368
Natural Born Killers (1994)              1.307198
Tank Girl (1995)                         1.277695
Rocky Horror Picture Show, The (1975)    1.260177
                                           ...   
Wrong Trousers, The (1993)               0.708666
Shawshank Redemption, The (1994)         0.700443
Great Escape, The (1963)                 0.692585
Rear Window (1954)                       0.688946
Close Shave, A (1995)                    0.667143
Name: ratings, Length: 1216, dtype: float64